## Live Disparity Map ##
<b>Description:</b> After you use the disparity map tuner to find the best values for the resulting image, then run this code to see a live view of your disparity map.
<hr>

#### Cell Block 1 ####
<b>Description:</b> This cell block just imports all the necessary libraries and loads up the necessary variables

In [ ]:
import cv2
import numpy as np
import traitlets
from IPython.display import display, HTML, clear_output
import ipywidgets.widgets as widgets
from jetcam.utils import bgr8_to_jpeg
from jetcam.usb_camera import USBCamera
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider, Button
from stereovision.calibration import StereoCalibrator
from stereovision.calibration import StereoCalibration
import os

camWidth=640
camHeight=480

cameraR = USBCamera(width=camWidth, height=camHeight, capture_device=0)
cameraL = USBCamera(width=camWidth, height=camHeight, capture_device=1)

imageR = cameraR.read()
imageL = cameraL.read()

cameraR.running = True
cameraL.running = True

image_widgetR = widgets.Image(format='jpeg')
image_widgetR.value = bgr8_to_jpeg(imageR)

image_widgetL = widgets.Image(format='jpeg')
image_widgetL.value = bgr8_to_jpeg(imageL)

button_layout = widgets.Layout(width='128px', height='64px')
button = widgets.Button(description="Take Picture", button_style='success', layout=button_layout)

camera_linkR = traitlets.dlink((cameraR, 'value'), (image_widgetR, 'value'), transform=bgr8_to_jpeg)
camera_linkL = traitlets.dlink((cameraL, 'value'), (image_widgetL, 'value'), transform=bgr8_to_jpeg)

print('done')

#### Cell Block 2 ####
<b>Description:</b> Here we import the calibration data

In [ ]:
# Implementing calibration data
print('Read calibration data...')
calibration = StereoCalibration(input_folder='calib_result')
print('done')

#### Cell Block 3 ####
<b>Description:</b> Finally, run a live view of the disparity map from a live feed of the stereo cameras.

In [ ]:
import time

SWS=5
PFS=5
PFC=29
MDS=-81
NOD=128
TTH=100
UR=6
SR=12
SPWS=120

dm_colors_autotune = True
disp_max = -100000
disp_min = 10000

def stereo_depth_map(rectified_pair):
    global disp_max, disp_min
    c, r = rectified_pair[0].shape
    disparity = np.zeros((c, r), np.uint8)
    sbm = cv2.StereoBM_create(numDisparities=16, blockSize=15)
    sbm.setPreFilterType(1)
    sbm.setPreFilterSize(PFS)
    sbm.setPreFilterCap(PFC)
    sbm.setMinDisparity(MDS)
    sbm.setNumDisparities(NOD)
    sbm.setTextureThreshold(TTH)
    sbm.setUniquenessRatio(UR)
    sbm.setSpeckleRange(SR)
    sbm.setSpeckleWindowSize(SPWS)
    dmLeft = rectified_pair[0]
    dmRight = rectified_pair[1]
    disparity = sbm.compute(dmLeft, dmRight)
    local_max = disparity.max()
    local_min = disparity.min()
    if (dm_colors_autotune):
        disp_max = max(local_max,disp_max)
        disp_min = min(local_min,disp_min)
        local_max = disp_max
        local_min = disp_min
    disparity_grayscale = (disparity-local_min)*(65535.0/(local_max-local_min))
    disparity_fixtype = cv2.convertScaleAbs(disparity_grayscale, alpha=(255.0/65535.0))
    disparity_color = cv2.applyColorMap(disparity_fixtype, cv2.COLORMAP_JET)
    return disparity_color

frameWidgetL = widgets.Image(format='jpeg', width=camWidth, height=camHeight)
frameWidgetR = widgets.Image(format='jpeg', width=camWidth, height=camHeight)

cameraBool = True

display(widgets.HBox([image_widgetR, image_widgetL]))
display(frameWidgetL)

output2 = widgets.Output()
display(output2)

while cameraBool:
    with output2:
        cameraL_frame = cameraL.value
        cameraR_frame = cameraR.value
        newFrameL = cv2.cvtColor(cameraL_frame, cv2.COLOR_BGR2GRAY)
        newFrameR = cv2.cvtColor(cameraR_frame, cv2.COLOR_BGR2GRAY)
        rectified_pair = calibration.rectify((newFrameL, newFrameR))
        disparity = stereo_depth_map(rectified_pair)
        frameWidgetL.value = bgr8_to_jpeg(disparity)
    time.sleep(0.001)